# Numerical Error — Monochromatic Wave Propagation

This notebook shows how to extract free surface time series from the numerical solution and use them to evaluate both dispersive and diffusive numerical errors.

We compare the performance of two numerical schemes:

1. **Conservative Staggered Scheme**  
   - 2nd-order flux reconstruction  
   - 2nd-order Runge–Kutta time integration

2. **HLLC Scheme**  
   - 5th-order flux reconstruction  
   - 3rd-order Runge–Kutta time integration

Both schemes are tested using three different grid resolutions, corresponding to **20**, **30**, and **40** grid points per wavelength.

## Imports

First, we need to set the working directory to the main WavePropError directory to import the `scripts` module.

We import `plotly.io as pio` to save the interactive Plotly plot as an HTML file. The `IPython.display.IFrame` is used to display this HTML file within the documentation, making the interactive plot accessible directly from the Sphinx-generated documentation.

In [1]:
import os

maindir = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
os.chdir(maindir)

from scripts import *
import plotly.io as pio
from IPython.display import IFrame

## User input

We define the wave parameters and numerical settings for the model runs, which will be used consistently across all model runs.

### Wave parameters

In [2]:
wave_amplitude = 0.5e-2  # wave amplitude in m, small amplitude assumption
wave_period = 2  # wave period in s
water_depth = 1  # water depth in m

wave_length = wavelength_nwogu(
    wave_period, water_depth
)  # wave length in m computed using Nwogu's dispersion relation
print("Wave length: {:.2f}".format(wave_length))

Wave length: 5.20


### Numerical settings

In [3]:
points_per_wl = [20, 30, 40]
domain_wl = 30
duration_tp = 60

courant_number = 0.5  # courant number

grid_size = [wave_length / ppwl for ppwl in points_per_wl]  # grid size in m
domain_size = wave_length * domain_wl
run_time = wave_period * duration_tp

for dx in grid_size:
    print("Grid size: {:.2f}".format(dx))
print("Domain size: {:.2f} m".format(domain_size))
print("Simulation time: {:.2f} s".format(run_time))

Grid size: 0.26
Grid size: 0.17
Grid size: 0.13
Domain size: 156.13 m
Simulation time: 120.00 s


Here, we specify the gauge locations, the time step for sampling time series, and the time interval used for error analysis.  
This ensures accurate results by excluding initial transients before the wave field stabilizes.

In [4]:
# We define equally spaced gauges between 15 and 20 wavelengths,
# with 10 gauges per wavelength based on the current grid resolution
gauges_location = []
for i in range(len(grid_size)):
    gauges = [
        x
        for x in np.linspace(
            15 * wave_length, 20 * wave_length, points_per_wl[i] * 5 + 1
        )
    ]
    gauges = np.array(gauges)
    gauges_location.append(gauges)

# We define the time interval for error computation
time_max = run_time
time_min = 50 * wave_period

## Model runs

We run the each scheme with the 3 different grid resolutions. For each scheme, the simulation results are stored in a list of datasets: ds_stagg and ds_hllc.

This example demonstrates how the `run_sine_wave` function can be used to generate multiple runs with varying grid sizes while keeping the wave parameters constant.

### Conservative Staggered Scheme - 2nd-order

In [5]:
ds_stagg = []

for i in range(len(grid_size)):

    ds = run_sine_wave(
        name_run="STAGG_GRID_{}".format(points_per_wl[i]),
        scheme=conservative_staggered,
        order_reconstruction=2,
        order_time_integration=2,
        courant_number=courant_number,
        grid_size=grid_size[i],
        domain_size=domain_size,
        water_depth=water_depth,
        wave_amplitude=wave_amplitude,
        wave_period=wave_period,
        run_time=run_time,
        output_interval=run_time,
        gauges_locations=gauges_location[i],
        gauges_dt=wave_period / 1000,
    )

    ds_stagg.append(ds)

Running STAGG_GRID_20...

Input parameters
----------------
Numerical scheme :conservative_staggered
Order of reconstruction :2
Order of RK time integration :2
Courant number :0.5
Grid spacing :0.260214 m
Length of the domain :156.128 m
Water depth :1 m
Total computation time :120 s
Time step for the output :120 s
Time step for the gauges :0.002 s
frequency [Hz]           amplitude [m]            phase [rad]              
0.5                      0.005                    0                        

Computation Progress:    
100% [||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||]

The numerical simulation took 2.63452 s


Reading STAGG_GRID_20 results and converting to xarray.Dataset...
Done!
Running STAGG_GRID_30...

Input parameters
----------------
Numerical scheme :conservative_staggered
Order of reconstruction :2
Order of RK time integration :2
Courant number :0.5
Grid spacing :0.173476 m
Length of the domain :156.128 m
Water depth :1 m
Total computation time :120 s
Time

### HLLC Scheme - fifth order reconstruction and third order time integration

In [6]:
ds_hllc = []

for i in range(len(grid_size)):

    ds = run_sine_wave(
        name_run="HLLC_GRID_{}".format(points_per_wl[i]),
        scheme=hllc,
        order_reconstruction=5,
        order_time_integration=3,
        courant_number=courant_number,
        grid_size=grid_size[i],
        domain_size=domain_size,
        water_depth=water_depth,
        wave_amplitude=wave_amplitude,
        wave_period=wave_period,
        run_time=run_time,
        output_interval=run_time,
        gauges_locations=gauges_location[i],
        gauges_dt=wave_period / 1000,
    )

    ds_hllc.append(ds)

Running HLLC_GRID_20...

Input parameters
----------------
Numerical scheme :hllc
Order of reconstruction :5
Order of RK time integration :3
Courant number :0.5
Grid spacing :0.260214 m
Length of the domain :156.128 m
Water depth :1 m
Total computation time :120 s
Time step for the output :120 s
Time step for the gauges :0.002 s
frequency [Hz]           amplitude [m]            phase [rad]              
0.5                      0.005                    0                        

Computation Progress:    
100% [||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||]

The numerical simulation took 5.10453 s


Reading HLLC_GRID_20 results and converting to xarray.Dataset...
Done!
Running HLLC_GRID_30...

Input parameters
----------------
Numerical scheme :hllc
Order of reconstruction :5
Order of RK time integration :3
Courant number :0.5
Grid spacing :0.173476 m
Length of the domain :156.128 m
Water depth :1 m
Total computation time :120 s
Time step for the output :120 s
Time step f

## Exact Solution

We compute a separate exact solution for each grid resolution to ensure consistency with the results fromat, as the functions used to compute dispersive and diffusive errors require both the exact and numerical solutions to be on the same grid.

In [7]:
ds_exact = []

for i in range(len(grid_size)):
    ds = sine_wave_exact_solution(
        water_depth=water_depth,
        wave_amplitude=wave_amplitude,
        wave_period=wave_period,
        time=ds_stagg[i].time.values,
        x=ds_stagg[i].x,
        time_gauges=ds_stagg[i].time_gauges.values,
        index_gauges=ds_stagg[i].index_gauges.values,
    )

    ds_exact.append(ds)

## Plotting

### Time series

We create an interactive plot of the time series for each scheme. The plot includes the exact solution and the numerical solutions for both schemes, allowing for a direct comparison of the results. We take the first resolution (20 grid points per wavelength) as an example, but the same process can be applied to the other resolutions as well.



In [ ]:
kk = 0  # second grid resolution, 20 points per wavelength

fig = plot_gauges(
    [ds_stagg[kk], ds_hllc[kk]],
    ds_exact[kk],
    [10 * wave_length, 15 * wave_length, 20 * wave_length],
)

# Additional formatting of the fig object
for i in range(1, 4):
    fig.update_yaxes(range=[-0.01, 0.01], row=i, col=1)

fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="center", x=0.5),
)

pio.write_html(fig, file="docs/source/_static/test3_time_series.html", auto_open=False)
IFrame(src="../_static/test3_time_series.html", width="100%", height="600px")

## Compute Numerical Error

In this section of the notebook, we compute the **dispersive** and **diffusive** errors for each numerical scheme and grid resolution. These errors are evaluated by comparing the numerical results from the model runs with the exact analytical solution.

The functions `compute_disp_error` and `compute_diff_error` are used to compute the dispersive and diffusive errors, respectively. or more details on the error calculations, please refer to the documentation of these functions.

The **diffusive error** is stored in the lists `rmse_amplitude_stagg` and `rmse_amplitude_hllc`, corresponding to the **conservative staggered scheme** and the **HLLC scheme**, respectively. Each element in these lists represents the root mean square error (RMSE) of the wave amplitude for a different grid resolution.

The **dispersive error** is stored in the lists `rmse_phase_stagg` and `rmse_phase_hllc`, corresponding to the **conservative staggered scheme** and the **HLLC scheme**, respectively. Each element in these lists represents the root mean square error (RMSE) of the wave phase for a different grid resolution.


### Diffusive error

In [9]:
rmse_amplitude_stagg = []
rmse_amplitude_hllc = []

for i in range(len(grid_size)):

    rmse_amplitude_stagg.append(
        diffusive_rmse(ds_stagg[i].sel(time_gauges=slice(time_min, time_max)))
    )

    rmse_amplitude_hllc.append(
        diffusive_rmse(ds_hllc[i].sel(time_gauges=slice(time_min, time_max)))
    )

In [10]:
print("RMSE amplitude for staggered scheme:")
for i in range(len(grid_size)):
    print(
        "Grid size: {:.2f} m, RMSE amplitude: {:.2e}".format(
            grid_size[i], rmse_amplitude_stagg[i]
        )
    )

print("\n")

print("RMSE amplitude for HLLC scheme:")
for i in range(len(grid_size)):
    print(
        "Grid size: {:.2f} m, RMSE amplitude: {:.2e}".format(
            grid_size[i], rmse_amplitude_hllc[i]
        )
    )

RMSE amplitude for staggered scheme:
Grid size: 0.26 m, RMSE amplitude: 2.28e-03
Grid size: 0.17 m, RMSE amplitude: 2.37e-03
Grid size: 0.13 m, RMSE amplitude: 1.87e-03


RMSE amplitude for HLLC scheme:
Grid size: 0.26 m, RMSE amplitude: 3.07e-01
Grid size: 0.17 m, RMSE amplitude: 7.84e-02
Grid size: 0.13 m, RMSE amplitude: 2.72e-02


### Dispersive error

In [11]:
rmse_phase_stagg = []
rmse_phase_hllc = []

for i in range(len(grid_size)):

    rmse_phase_stagg.append(
        dispersive_rmse(
            ds_stagg[i].sel(time_gauges=slice(time_min, time_max)),
            ds_exact[i].sel(time_gauges=slice(time_min, time_max)),
        )
    )

    rmse_phase_hllc.append(
        dispersive_rmse(
            ds_hllc[i].sel(time_gauges=slice(time_min, time_max)),
            ds_exact[i].sel(time_gauges=slice(time_min, time_max)),
        )
    )

In [12]:
print("RMSE phase for staggered scheme:")
for i in range(len(grid_size)):
    print(
        "Grid size: {:.2f} m, RMSE phase: {:.2e}".format(
            grid_size[i], rmse_phase_stagg[i]
        )
    )

print("\n")

print("RMSE phase for HLLC scheme:")
for i in range(len(grid_size)):
    print(
        "Grid size: {:.2f} m, RMSE phase: {:.2e}".format(
            grid_size[i], rmse_phase_hllc[i]
        )
    )

RMSE phase for staggered scheme:
Grid size: 0.26 m, RMSE phase: 6.54e-02
Grid size: 0.17 m, RMSE phase: 2.99e-02
Grid size: 0.13 m, RMSE phase: 1.77e-02


RMSE phase for HLLC scheme:
Grid size: 0.26 m, RMSE phase: 9.26e-02
Grid size: 0.17 m, RMSE phase: 2.78e-02
Grid size: 0.13 m, RMSE phase: 1.01e-02
